# Assignment 1
## Getting started with SED fitting & error estimation


This assignment is split into 3 sections, roughly corresponding to the contents of each of the 3 weeks in the Error Estimation module. 

The SETUP section is designed to be done first, to familarize yourself with the data and the details of SED fitting. Section 1 is shorter to account for this. You should be able to finish Section 1 after January 26th, Section 2 after February 2nd, and Section 3 after February 9th. You can work on this assignment during the lab hours, as well as in your own time - and feel free to ask the instructors questions!

All assignments are presented as Jupyter notebooks. You will fork the repository to have your own access to all files. You can edit this notebook directly with your answers and push your changes to GitHub. Feel free to write any commonly used functions in a separate module and import them here if you like.

# Section 0
## Setup

First, we need to set up the SED fitting project, install Prospector, and make sure we can run Prospector.

1. Download the data from [here](https://irfu.cea.fr/Pisp/yu-yen.chang/sw.html) (both the input and output catalog). <br>
   The data comes from [Chang et al. (2015)](https://ui.adsabs.harvard.edu/abs/2015ApJS..219....8C/abstract), who used MAGPHYS (another SED fitting code) to fit the photometry stored in the input file and obtain stellar masses / ages / etc. stored in the output file. <br>
   > Be careful of where you store these files. If you store them in a random place in this repository, they will get pushed to Git alongside your changes. The files are heavy, which will cause problems downstream. We suggest placing your files in `projects/sed-fitting/data` folder (for Chang catalogs) and `projects/sed-fitting/output` folder (for your own Prospector output files). These folders are added in the `.gitignore` file in the root, and so anything in these folders will be ignored by Git.
   
2. Install [Prospector](https://github.com/bd-j/prospector)
   > We found this easiest to do using the [conda script](https://github.com/bd-j/prospector/blob/main/conda_install.sh) provided: download the script somewhere on your PC and run `bash conda_install.sh`. Again, best not to do this inside the Git repository as the bash script will download several large libraries. If you are feeling brave, you can try doing it from this repository, but first make sure to add the libraries to `.gitignore`. <br>
   > Make sure to follow the instructions from `conda_install.sh`: add SPS_HOME to your `~/.bashrc` file as your terminal says.<br>
   > This bash script will create a new conda environment, `prospector`, which you can run using `conda activate prospector`.
    <br><br>

3. Fix a Prospector + NumPy >= 1.20 issue (you might not need this)<br>
   Run the following cell:

In [5]:
import numpy
print(numpy.__version__)

1.26.0


If the version is greater than 1.20.0, you will need to fix a library in Prospector to make it compatible with a later NumPy. There is a file `write_results.py` in the project library. Copy it to your Prospector installation location within your conda environment, it should be something like `/home/[username]/[your conda installation]/envs/prospector/lib/[your python version]/site-packages/prospect/io`

4. Install any other Python libraries<br>Since we have made a new environment, it will only have prospector and its dependancies installed. We still need to install other useful Python libraries in the new environment. 
   > We listed the libraries you will need in `requirements.txt`.<br>
   > First, run `conda activate prospector` to make sure you are using the new environment. <br>
   > Then, you can install all required libraries simply using `pip install -r requirements.txt`


## Test Prospector

We have provided a notebook, Prospector Example, that gives step-by-step instructions for loading in the photometric data for one galaxy and running Prospector to get a simple fit to that galaxy's SED. If you have any questions, you can either ask us, or consult the [Prospector documentation](https://prospect.readthedocs.io/en/latest/) that has explanations of all the different parameters, models, and use cases.

#### Step 1
Open the example notebook and run it completely without changing anything. <br> (This is to make sure everything is installed right)

#### Step 2
Try to change some fitting parameters: you can try a different galaxy, play around with different priors, try a different star formation model, or a different sampling technique.

What did you try? What was different between the fits?

#### Step 3
Try running Prospector through a command line.

Re-running a notebook each time is impractical, so to fit several galaxies, it's easier to write everything as a Python script we can run from the command line. We wrote a Python script for you, `assignment_params.py`. 

It has a few command line arguments. You can see the meaning of the different arguments by running

     python assignment_params.py --help

Try running this line in your command line to ensure that Prospector works fine and produce a fit for a test galaxy.

> Note: if you stored your data files elsewhere, you will need to change the path to the input catalog on Line 200.

#### Step 4
Plot the fitting results.

In [11]:
import prospect.io.read_results as reader
import numpy as np
import matplotlib.pyplot as plt

In [12]:
file_name = 'output/test_24Jan10-12.05_result.h5'
res, obs, model = reader.results_from(file_name)
results_type = "emcee" # | "dynesty"
randint = np.random.randint

sps = reader.get_sps(res)


if results_type == "emcee":
## ANI NOTES you can choose this at random?
    
    nwalkers, niter = 2, 2
    theta = res['chain'][randint(nwalkers), randint(niter)]
else:
    theta = res["chain"][randint(len(res["chain"]))]

imax = np.argmax(res['lnprobability'])

if results_type == "emcee":
    i, j = np.unravel_index(imax, res['lnprobability'].shape)
    theta_max = res['chain'][i, j, :].copy()
    thin = 5
else:
    theta_max = res["chain"][imax, :]
    thin = 1

a = 1.0 + model.params.get('zred', 0.0) # cosmological redshifting
# photometric effective wavelengths
wphot = obs["phot_wave"]
# spectroscopic wavelengths
# *restframe* spectral wavelengths, since obs["wavelength"] is None
wspec = sps.wavelengths
wspec *= a #redshift them
xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
initial_spec, initial_phot, initial_mfrac = model.sed(theta, obs=obs, sps=sps)

temp = np.interp(np.linspace(xmin,xmax,10000), wspec, initial_spec)
ymin, ymax = temp.min()*0.8, temp.max()/0.4


In [23]:
initial_spec, initial_phot, initial_mfrac = model.sed(theta, obs=obs, sps=sps)

# generate model
prediction = model.mean_model(theta, obs=obs, sps=sps)
pspec, pphot, pfrac = prediction

#### Step 5

The code below loads in the table containing the photometry of all galaxies as a [Pandas DataFrame](https://pandas.pydata.org/). Select a sample of galaxies to run Prospector on. 

In [3]:
from astropy.table import Table
from astropy.io import fits
with fits.open('../data/sw_input.fits') as f:
    df = Table(f[1].data).to_pandas()
    f.close()

df.head()

,id,ra,dec,redshift,PLATE,MJD,FIBERID,designation,flux0_u,flux0_u_e,...,flux_w2_e,flux_w3,flux_w3_e,flux_w4,flux_w4_e,extin_u,extin_g,extin_r,extin_i,extin_z
0,3,337.450310,1.266134,0.088372,376,52143,404,J222948.07+011558.1,0.000031,0.000003,...,0.000049,4.172000e-07,0.000209,0.000002,0.001187,0.341327,0.265960,0.183990,0.136724,0.101698
1,5,338.115522,1.270146,0.163800,376,52143,567,J223227.69+011612.6,0.000011,0.000004,...,0.000111,9.851000e-07,0.000493,0.000004,0.001883,0.368063,0.286793,0.198402,0.147434,0.109664
2,8,341.101481,1.266255,0.143369,378,52146,404,J224424.38+011558.3,0.000017,0.000003,...,0.000039,1.013700e-06,0.000507,0.000008,0.003856,0.337630,0.263079,0.181997,0.135243,0.100596
3,12,341.870909,1.267913,0.275242,378,52146,567,J224729.01+011604.3,0.000007,0.000003,...,0.000106,9.999000e+00,9.999000,0.000009,0.004526,0.358145,0.279064,0.193055,0.143461,0.106709
4,14,342.686706,1.270160,0.089104,676,52178,373,,0.000031,0.000003,...,9.999000,9.999000e+00,9.999000,9.999000,9.999000,0.362289,0.282293,0.195289,0.145121,0.107943


**Optional:** how did you choose your set of galaxies? It's okay if they are all random, but if you want, you can try to choose a sample you find interesting.

In [ ]:
# Space to select a sample of N galaxies, answer the question, or do something else

#### Step 6

Run Prospector on your chosen sample. Feel free to use the default code we provided, or try your own fitting / modelling parameters.

#### Step 7

The photometry of the galaxy changes based on how far away it is: galaxies further away are fainter, and their SED shifts more towards red wavelengths. This is one of the challenges of SED fitting: it's hard to break the degeneracy between galaxies that are red because they are intrinsically red (older stars), and galaxies that are red because they are far away. 

By default, we give Prospector a *known* spectroscopic redshift for each galaxy. This means that it is not fitting one of the important degenarate parameters.

However, obtaining a spectroscopic redshift is difficult, as it requires measuring a galaxy spectrum - simple photometry is way easier. The galaxies we used in this project all have spectroscopy, but there are millions of galaxies that do not. For those objects, we need to fit the SED *and* redshift simultaneously.

Let's imagine that we don't know the redshift of our objects. For your sample, re-run the Prospector fits by setting redshift as one of the free parameters. You can change Lines 124-130 of `assignment_params.py` to allow redshift to vary.

# Section 1

- #### Goodness of fit of Prospector examples

In [ ]:
### Plot the data for a chosen galaxy (with error bars) 
### Flux or magnitude vs band wavelength or index

### Add the best fit model to the plot

### Compute the goodness-of-fit (chi squared)
### (Prospector assumes the magnitudes are independent so you can to, but we'll come back to this later)

In [ ]:
### Is the goodness of fit reasonable (why?)

In [ ]:
### What is the best fit, mean, and 1/2/3 sigma confidence intervals for each of the constrained parameters
### Are they consistent with the results from Chang et al (the output file linked above)?
### How similar do we expect them to be?

In [ ]:
### How did allowing the redshift as a free parameter change the results? did you get the same mass? is the redshift correct?

# SECTION 2 

- #### Covariance of contrained parameters (Gaussian assumption)

In [ ]:
### First lets look at the covariance of constrained parameters
### Plot a corner plot of the prospector outputs, showing the 68% and 95% 2D contours
### (You will want to use one of the MCMC methods in the prospector fitting ...
###    we will discuss this more in the MCMC section. For now we can assume that the density of ...
###    output samples at a given location in parameter space, is proportional to the probability of ...
###    those parameters, given the data and model )

In [ ]:
### Are there any degeneracies between parameters in the fit?
### What does this mean?

In [ ]:
### Make a covarinace matrix of the fitted parameters (describing the uncertainties and their covarinace with each other) 
### Plot it
### Looking at the contour plot, was this a reasonable thing to do?

- #### Covariance of magnitude errors

In [ ]:
### So far, prospector has assumed the uncertainties in the magnitudes/fluxes are independent of each other 
### In practice this might not be true
### For this excercise, assume the correlation between the flux in each band is X%
### Plot the covariance matrix, with and without the correlated errors

In [ ]:
### Re-compute the goodness of fit with the correlated errors

In [ ]:
### Did the goodness-of-fit get worse or better, why?

# SECTION 3 

Estimating error bars and uncertainty of distributions (shot noise, bootstrap)

In [ ]:
### For this section we will use the output catalogs, since running prospector on all 
### 800000 galaxies would be a waste of computing for this class

### Plot a histogram of a given measured quantity (stellar mass, redshift, etc) 
### Choose the range and bin size appropriately so that we can see the full distribution

In [ ]:
### There are a limited number of objects in each histogram bin
### Add shot noise (Poisson) to the histogram bars to show this
### These with be your "analytic" error bars 

In [ ]:
### Now split the data into N subsets and compute the jackknife covarinace of the histogram bins 
### How does it compare to the analytic errors
### Are the bins independent?
### What if you make N very large or very small


In [ ]:
### Now repeat this excercise, replacing the histogram with a calculation of the mean stellar 
### mass as a function of redshift (i.e. Split the data into redshift bins, and compute the mean mass in each)
### Use Jackknife to get the errors
### How to these compare with the standard error of the mean?